### Retrieval-Augmented Generation with Wikipedia

In [1]:
import os
import openai
from gpt_helper import GPTchatClass, printmd
from wiki_helper import wiki_search
from util import printmd, extract_quoted_words

print("openai version:[%s]" % (openai.__version__))

openai version:[1.3.7]


### Instantiate GPT Agent

In [2]:
GPT = GPTchatClass(
    gpt_model="gpt-3.5-turbo",  # 'gpt-3.5-turbo' / 'gpt-4'
    role_msg="Your are a helpful assistant summarizing infromation and answering user queries.",
)

key_path:[../key/rilab_key.txt]

Chat agent using  initialized with the follow role:[Your are a helpful assistant summarizing infromation and 
answering user queries.]

### Our RAG agent will use the following strategies
We assume that a user question is given (e.g., 'Who is the current president of South Korea?').
* Step 1. For the given question, our `GPT agent` will first generate a number of entities for searching Wikipedia.
* Step 2. Then, our `WikiBot` will provide (i.e., crawl) related information summarized with the `GPT agent` considering the user question.
* Step 3. Finally, the summarized texts and the original user question will be given to the `GPT agent` to answer. 

In [3]:
question = 'Who is the current president of South Korea?'
"""
question = '''
    I am an interactive humanoid robot agent. 
    I have following action capabilites:['idle','waving','greeting','raising hands','hugging','reading a book']
    I can detect following observations:['no people','a person appears','a person waves hands','a person leaves']
    I have a following personality:['Introverted and Childish']
    What is the best next action when I am in ['idle'] state and observes ['a person waves hands']?
'''
"""
print ("question: %s"%(question))

question: Who is the current president of South Korea?


### Step 1. Generate entities for wiki search

In [4]:
user_msg = \
    """
    Suppose you will use Wikipedia for retrieving information. 
    Could you recommend three query words wrapped with quotation marks considering the following question?
    """ + '"' + question + '"'

In [5]:
response_content = GPT.chat(
    user_msg=user_msg,PRINT_USER_MSG=True,PRINT_GPT_OUTPUT=True,
    RESET_CHAT=True,RETURN_RESPONSE=True)

[USER_MSG]


    Suppose you will use Wikipedia for retrieving information. 
    Could you recommend three query words wrapped with quotation marks considering the following question?
    "Who is the current president of South Korea?"

[GPT_OUTPUT]

Sure! Here are three query words wrapped with quotation marks that you can use to search for information on the current president of South Korea:

1. "current president of South Korea"
2. "president of South Korea"
3. "South Korea president"

In [6]:
# Print summarized sentence with a markdown format
printmd(response_content)

Sure! Here are three query words wrapped with quotation marks that you can use to search for information on the current president of South Korea:

1. "current president of South Korea"
2. "president of South Korea"
3. "South Korea president"

In [7]:
entities = extract_quoted_words(response_content)
if len(entities) > 3:
    entities = entities[-3:]
print(entities)

['current president of South Korea', 'president of South Korea', 'South Korea president']


### Step 2. Query entities to `WikiBot`

In [8]:
paragraphs_return = []
for entity in entities:
    paragraphs_return += wiki_search(entity=entity, VERBOSE=True)

entity:[current president of South Korea] mismatched. use [President of South Korea] instead.
 We have total [294] paragraphs.
 After filtering, we have [31] and [8] paragraphs returned (k:[5] and m:[3])
entity:[president of South Korea] matched.
 We have total [294] paragraphs.
 After filtering, we have [31] and [8] paragraphs returned (k:[5] and m:[3])
entity:[South Korea president] matched.
 We have total [294] paragraphs.
 After filtering, we have [31] and [8] paragraphs returned (k:[5] and m:[3])


In [9]:
# Get the unique elements
paragraphs_unique = list(set(paragraphs_return))
print(
    "Number of paragraphs [%d] => unique ones [%d]"
    % (len(paragraphs_return), len(paragraphs_unique))
)

Number of paragraphs [24] => unique ones [8]


In [10]:
# Now summarize each paragraph into a single sentence considering the question
summarized_sentences = []
for p_idx, p in enumerate(paragraphs_unique):
    user_msg = "You are given following question: " + question
    user_msg += "Could you summarize the following paragraph into one setence? \n " + p
    response_content = GPT.chat(
        user_msg=user_msg,
        PRINT_USER_MSG=False,
        PRINT_GPT_OUTPUT=False,
        RESET_CHAT=True,
        RETURN_RESPONSE=True,
    )
    # Append summarized sentences
    summarized_sentences.append(response_content)
    # Print summarized sentence with a markdown format
    printmd(response_content)

The current president of South Korea is directly elected for a five-year term with no possibility of re-election and is exempt from criminal liability, except for insurrection or treason.

The current president of South Korea serves a five-year term and is not eligible for re-election.

The current president of South Korea is the head of state and government, leading the State Council and serving as the commander-in-chief of the Republic of Korea Armed Forces.

The current president of South Korea does not have the power to dissolve the National Assembly and can take emergency measures that may amend or abolish laws, but these measures must be endorsed by the National Assembly to be in effect.

The current president of South Korea chairs the National Security Council and there is also a Peaceful Unification Advisory Council, which serves as a government sounding board and provides opportunities to meet with senior officials.

The controversial Advisory Council of Elder Statesmen, which was expanded and elevated to cabinet rank, was intended to preserve the status and position of a former president but plans were announced to reduce its size and functions after the inauguration of the current president.

The current president of South Korea, Yoon Suk Yeol, assumed office on 10 May 2022 after winning the 2022 presidential election with 48.5% of the vote.

The legitimacy of the Provisional Government established in 1919 and its continuity has been recognized and succeeded by South Korea in its constitutions of 1948 and 1988.

### Step 3. Answer the question using `summarized_sentences`

In [11]:
user_msg = " ".join(summarized_sentences)
user_msg += " Using the information above, could you answer the following question? "
user_msg += question

In [12]:
response_content = GPT.chat(
    user_msg=user_msg,
    PRINT_USER_MSG=True,
    PRINT_GPT_OUTPUT=True,
    RESET_CHAT=False,
    RETURN_RESPONSE=True,
)

[USER_MSG]

The current president of South Korea is directly elected for a five-year term with no possibility of re-election and is exempt from criminal liability, except for insurrection or treason. The current president of South Korea serves a five-year term and is not eligible for re-election. The current president of South Korea is the head of state and government, leading the State Council and serving as the commander-in-chief of the Republic of Korea Armed Forces. The current president of South Korea does not have the power to dissolve the National Assembly and can take emergency measures that may amend or abolish laws, but these measures must be endorsed by the National Assembly to be in effect. The current president of South Korea chairs the National Security Council and there is also a Peaceful Unification Advisory Council, which serves as a government sounding board and provides opportunities to meet with senior officials. The controversial Advisory Council of Elder Statesmen, which was expanded and elevated to cabinet rank, was intended to preserve the status and position of a former president but plans were announced to reduce its size and functions after the inauguration of the current president. The current president of South Korea, Yoon Suk Yeol, assumed office on 10 May 2022 after winning the 2022 presidential election with 48.5% of the vote. The legitimacy of the Provisional Government established in 1919 and its continuity has been recognized and succeeded by South Korea in its constitutions of 1948 and 1988. Using the information above, could you answer the following question? Who is the current president of South Korea?

[GPT_OUTPUT]

The current president of South Korea is Yoon Suk Yeol.

In [13]:
user_msg = "Could you explain about this little longer?"
response_content = GPT.chat(
    user_msg=user_msg,
    PRINT_USER_MSG=True,
    PRINT_GPT_OUTPUT=True,
    RESET_CHAT=False,
    RETURN_RESPONSE=True,
)

[USER_MSG]

Could you explain about this little longer?

[GPT_OUTPUT]

Certainly! Yoon Suk Yeol is the current president of South Korea. He assumed office on 10th May 2022 after winning the 2022 presidential election with 48.5% of the vote. Yoon Suk Yeol is a former prosecutor general and member of the conservative People Power Party. As the president, he serves a five-year term and is not eligible for re-election. He is also exempt from criminal liability, except for insurrection or treason. As the head of state and government, Yoon Suk Yeol leads the State Council and acts as the commander-in-chief of the Republic of Korea Armed Forces. He chairs the National Security Council and also oversees the Peaceful Unification Advisory Council, a government advisory board. It is important to note that the president does not have the power to dissolve the National Assembly, and any emergency measures taken by the president must be endorsed by the National Assembly to be in effect. The Provisional Government established in 1919 holds legitimacy and continues to have its role recognized in the constitutions of South Korea.